In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_3_5 = "gpt-3.5-turbo-1106"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_3_5

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

In [2]:
SYSTEM_PROMPT="""
I want you to act as a software engineer.
Your task is to read this RSL Specifcation and turn it to a NL Specification.

Example of a NL Specification:
Review test
Main flow:
Step 1: Learners select the review button on the Home page. 
Step 2: The system redirects learners to the review's screen.
Step 3: System randomly selects one of four methods including questions, sound, image or meanings of one word for each words in learner's daily review words list with four selections of answer in the bottom of the screen. 
Step 4: Learners chooses by clicking on one of four selections.
Step 5: Learners click the right answer. 
Step 6: System shows a green word for learners to read. 
Step 7: This word is increased by the system in memory level.
Step 8: Learners press "Continue" button to go to the next word in the review words list.

Alternative flow 1: Learners choose the wrong answer
At step 5 of the basic flow: Learners click the wrong answer. 
Step 6: System shows a red word for learners to read. . 
Step 7: This word is decreased by the system in memory level.
Go back to step 8 in the basic flow and continue with the steps from step 8.

Alternative flow 2: Learners have done the review by coming to the last word in the review list
At step 8 of the basic flow: Learners press "Continue" button to go back to the Home page.
Step 9: System saved the review.

Alternative flow 3: Learners press "Exit" button and choose to stay
At step 4 of the basic flow: Learners press "Exit" button.
Step 5: System displays a pop up with two buttons "Confirm" and "Stay". 
Step 6: Learners choose "Stay".
Go back to step 4 in the basic flow and continue with the steps from step 4

Exception flow 1: Learners press "Exit" button and choose to exit
At step 3 of the basic flow: Learners press "Exit" button.
Step 4: System displays a pop up with two buttons "Confirm" and "Stay". 
Step 5: Learners choose "Confirm". 
Step 6: System redirect learners to Home page and the review will not be saved.

Pre condition:
User has login as learner. In their account, there are some words has been learned so there will be a review words list for today.

Post condition:
Update the status of vocabulary (memory level, review time) corresponding to the results of exercise checking. The status of the vocabulary to be reviewed is changed. The done review is saved and user could look back.

Rules for you to remember:
- Extract flow and their descriptions into distinct paragraph.
- Please use the same word as the example like "At step" or "Go back" if there are sign of changing flow.
"""

In [3]:
user_prompt="""
 1 /********************************************************
 2 UseCase
 3 *********************************************************/
 4 UseCaseuc_1_Signup"Signup":EntityCreate[
 5 primaryActor a_Anonymous
 6 dataEntity e_User
 7 precondition "TheUseremailcannotbealreadyregisteredinthesystem."
 8 postcondition"Thenewuseriscreatedin thesystem"
 9 description"AsanAnonymoususerIwantto signuptobecome auser"
 10
 11 mainScenario uc_1_SignUp_MainScenario(Main)"AnonymousUser SignsUpWithhis/her
 Information"[
 12 steps1(System:Execute:Open)"Opensthesignuppageinthe browser"
 13 steps2(Actor:PrepareData:Fill)"Fillsinthename"
 14 steps3(Actor:PrepareData:Fill)"Fillsintheemail"[
 15 scenarios3a_AnEmailAlreadyExistsInTheSystem(Exception)"AnonymousUserSignsUpWithAn
 EmailThatAlreadyExistsInTheSystem"[
 16 steps3a1(System:Execute:Check)"Validatesifthesubmittedemaildoesnotexists"
 17 steps3a2(System:ReturnResult:DisplayMessage)"Thesubmittedemaildoesnotexists"
 nextSteps3]]
 18 steps4(Actor:PrepareData:Fill)"Fillsinthepassword"
 19 steps5(Actor:PrepareData:Fill)"Fillsintheconfirmpassword"
 20 steps6(Actor:CallSystem:SelectAction) "Selectstheaction signup"[
 21 scenarios6a_SignUpWithDifferentPasswords(Exception)"AnonymousUserSignsUpWith
 DifferentPasswords"[
 22 steps6a1(System:Execute:Check)"Validatesthepasswordsaredifferent"
 23 steps6a2(System:ReturnResult:DisplayMessage)"Thepasswordsaredifferent"nextSteps6
 ]]
 24 steps7(System:Execute:Check)"Validates thepasswordsare correct"
 25 steps8(System:ReturnResult:DisplayMessage)"Theuserwassuccessfullycreatedinthesystem"
 26 ]
 27 ]
"""

In [4]:
user_prompt="""
1 UseCase uc_3_CreateProduct"CreateProduct":EntityCreate[
 2 primaryActora_Manager
 3 dataEntitye_Product
 4 precondition"Theuserisauthenticatedasamanager"
 5 postcondition"Theproductiscreatedinthesystem"
 6 description"AsamanagerIwanttocreateaproduct"
 7
 8 mainScenariouc_3_CreateProduct_MainScenario(Main) "Usercreatesaproduct"[
 9 steps1(System:Execute:Open)"Opens theproductslistpageinthebrowser"
 10 steps2(Actor:CallSystem:SelectAction)"Selectsthe actionopenproductpage"
 11 steps3(Actor:PrepareData:Fill)"FillsintheproductID"[
 12 scenarios3a_ProductIdAlreadyExists(Exception)"UserfillsinaproductIDthatalready
 exists"[
 13 steps3a1(System:Execute:Check)"Validatestheproduct IDalreadyexists"
 14 steps3a2(System:ReturnResult:DisplayMessage)"TheproductIDalreadyexists"nextStep
 s3]
 15 scenarios3b_NotFillProductIdAndTheSystemAutoGeneratedIt(Alternative)"Doesnotfillin
 theproductIDandtheSystemauto−generatedit"[
 16 steps3b1(System:Execute:Check)"Validatestheproduct IDisempty"
 17 steps3b2(System:Execute:SaveData)"Auto−generatesthe productID"nextSteps4]]
 18 steps4(Actor:PrepareData:Fill)"Fillsintheproductname"[
 19 scenarios4a_DoesNotFillInTheProductName(Exception) "Userremovestheproductname"[
 20 steps4a1(System:Execute:Check)"Validatestheproduct nameisempty"
 21 steps4a2(System:ReturnResult:DisplayMessage)"Theproductnameisempty"nextSteps4]
 ]
 22 steps5(Actor:CallSystem:SelectAction)"Selectsthe actiontocreatetheproduct"
 23 steps6(System:ReturnResult:DisplayMessage)"Theproductwassuccessfullycreated"
 24 ]
 25 ]
"""

In [5]:
promptTestCase = [
{ "role": "system", "content": SYSTEM_PROMPT},
{ "role": "user", "content": user_prompt}
]
gpt_response = ask(promptTestCase, client, model)


In [6]:
print(gpt_response)

Specification: Create Product

As a manager, I want to create a product.

Main Flow:
At step 1: The manager opens the products list page in the browser.
At step 2: The manager selects the action to open the product page.
At step 3: The manager fills in the product ID.
  - Exception: User fills in a product ID that already exists.
    - At step 3a1: The system validates the product ID already exists.
    - At step 3a2: The system displays a message "The product ID already exists" and proceeds to step 3.
  - Alternative: User does not fill in the product ID and the system auto-generates it.
    - At step 3b1: The system validates the product ID is empty.
    - At step 3b2: The system auto-generates the product ID and proceeds to step 4.
At step 4: The manager fills in the product name.
  - Exception: User removes the product name.
    - At step 4a1: The system validates the product name is empty.
    - At step 4a2: The system displays a message "The product name is empty" and proceeds to